In [1]:
import sys
import time

sys.path.append('../../..')

In [4]:
from omegaconf import OmegaConf
from together import Together
from transformers import AutoTokenizer
from tqdm import tqdm

from src.data_prepocessing import load_ds
from src.evaluation import Evaluator

# Experiment set up

In [5]:
config = OmegaConf.load("llama_config.yaml")
print(OmegaConf.to_yaml(config))

model_name: meta-llama/Llama-3.3-70B-Instruct-Turbo
prompt: Ты професиональный разметчик с лингвистическим образованием. Внимательно прочитай
  диалог. Перепишите неполное высказывание на основе истории диалога. Ответ должен
  содержать только переписанное высказывание



# Data loading

In [6]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.3-70B-Instruct")

In [7]:
ds = load_ds("2rca_checked_version.json")

In [8]:
ds['train'][0]

{'history': ['Привет.',
  'Привет.',
  'Как твоё настроение сегодня?',
  'Настроение отлично, скоро куплю машину, а твоё настроение как?',
  'Здорово, ты скоро купишь машину! У меня тоже настроение на позитиве! Я очень давно хочу в Бразилию, и, кажется, сегодня моя мечта сбудется! Вечером поедем с подругой в турфирму оплачивать наш тур в Бразилию. А у тебя есть мечта?',
  'Давно хочу завести лабрадора, прям с детства хочу собаку. Может, когда дочка подрастет заведу себе лабрадора.',
  'Моей собаке уже 5 лет, и я даже не представляю, как я могла жить без своей собаки раньше?! Я думаю, что у тебя всё получится и у вас скоро обязательно появится питомец! Ведь собаки такие милые! Что сегодня будешь готовить на ужин?'],
 'phrase': 'Сегодня будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.',
 'rewrite': 'Сегодня на ужин будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'}

In [ ]:
def infer_ds(ds, model, **kwargs):
    test_results = []

    for i in tqdm(range(100)):
        success = False
        while not success:
            try:
                messages = [
                    {
                        "role": "system",
                        "content": config.prompt
                    },
                    {
                        "role": "user",
                        "content": "История: " + str(ds['test']['history'][i][-1]) + " Неполное высказвание: " + ds['test']["phrase"][i]
                    }
                ]
                response = kwargs["client"].chat.completions.create(
                    model=model, 
                    messages=messages,
                    temperature=1
                )
                out = response.choices[0].message.content
                test_results.append(out)
                success = True
            except:
                time.sleep(20)

    return test_results

In [ ]:
client = Together()

evaluator = Evaluator(dataset=ds, 
                      model=config.model_name, 
                      client=client,
                      tokenizer=tokenizer, 
                      infer_func=infer_ds)

evaluator.evaluate()

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,46.458949,0.575102,0.445694,0.354053,0.281528,0.572068,0.234446,0.165755,0.13573,0.119438
